In [ ]:
pwd

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
from astropy.io import fits
import astropy.units as u
import hcipy as hc
import matplotlib as mpl
os.chdir('/Users/asahoo/repos/PASTIS/')

import pastis.util as util
from pastis.config import CONFIG_PASTIS

from pastis.e2e_simulators.luvoir_imaging import LuvoirAPLC
from pastis.e2e_simulators.luvoir_imaging import LuvoirA_APLC

In [ ]:
ls

In [ ]:
nseg = 120            #for luvoir?
wvln = 638e-9

datadir = '/Users/asahoo/laurent_repos/PASTIS/LUVOIR_delivery_May2019/' #confused what this directory actually does?
aper_path = 'inputs/TelAp_LUVOIR_gap_pad01_bw_ovsamp04_N1000.fits'
aper_ind_path = 'inputs/TelAp_LUVOIR_gap_pad01_bw_ovsamp04_N1000_indexed.fits'
aper_read = hc.read_fits(os.path.join(datadir, aper_path))
aper_ind_read = hc.read_fits(os.path.join(datadir, aper_ind_path))

pupil_grid = hc.make_pupil_grid(dims=aper_ind_read.shape[0], diameter=15)

aper = hc.Field(aper_read.ravel(), pupil_grid)
aper_ind = hc.Field(aper_ind_read.ravel(), pupil_grid)

wf_aper = hc.Wavefront(aper, wvln)
hdr = fits.getheader(os.path.join(datadir, aper_ind_path))

poslist = []
for i in range(nseg):
    segname = 'SEG' + str(i+1)
    xin = hdr[segname + '_X']
    yin = hdr[segname + '_Y']
    poslist.append((xin, yin))
    
poslist = np.transpose(np.array(poslist))
seg_pos = hc.CartesianGrid(hc.UnstructuredCoords(poslist))
eunit = 1e-9

In [ ]:
conda env list

In [ ]:
print(type(pupil_grid), pupil_grid.shape, aper_ind_read.shape[0])

In [ ]:
plt.figure(figsize=(20,5))

plt.subplot(1,3,1)
plt.title("Pupil_Grid")
plt.plot(pupil_grid.x, pupil_grid.y, '+')
#plt.axis('equal')
plt.xlabel('x')
plt.ylabel('y')

plt.subplot(1,3,2)
plt.title("aper")
hc.imshow_field(aper)

plt.subplot(1,3,3)
plt.title("aper_ind")
hc.imshow_field(aper_ind)
plt.colorbar()

plt.show()

In [ ]:
plt.figure(figsize=(20,5))

plt.subplot(1,3,1)
plt.title("wf_aber.intensity")
hc.imshow_field(wf_aper.intensity)

plt.subplot(1,3,2)
plt.title("wf_aber.phase")
hc.imshow_field(wf_aper.phase)

plt.subplot(1,3,3)
plt.title("seg_pos")
plt.plot(seg_pos.x, seg_pos.y, '+')
#plt.axis('equal')
plt.xlabel('x')
plt.ylabel('y')

In [ ]:
hdr

In [ ]:
savedpath = '/Users/asahoo/Desktop/data_repos/laurent_data/2021-02-10T13-01-44_luvoir-medium'
G_LO_real = fits.getdata(os.path.join(savedpath, 'matrix_numerical', 'EFIELD_Re_matrix_num_LO_15.fits'))
G_LO_imag = fits.getdata(os.path.join(savedpath, 'matrix_numerical', 'EFIELD_Im_matrix_num_LO_15.fits'))
G_MID_real = fits.getdata(os.path.join(savedpath, 'matrix_numerical', 'EFIELD_Re_matrix_num_MID_6.fits'))
G_MID_imag = fits.getdata(os.path.join(savedpath, 'matrix_numerical', 'EFIELD_Im_matrix_num_MID_6.fits'))
G_HI_real = fits.getdata(os.path.join(savedpath, 'matrix_numerical', 'EFIELD_Re_matrix_num_HI_24.fits'))
G_HI_imag = fits.getdata(os.path.join(savedpath, 'matrix_numerical', 'EFIELD_Im_matrix_num_HI_24.fits'))
G_LOWFS_real = fits.getdata(os.path.join(savedpath, 'matrix_numerical', 'EFIELD_LOWFS_Re_matrix_num_LO_15.fits'))
G_LOWFS_imag = fits.getdata(os.path.join(savedpath, 'matrix_numerical', 'EFIELD_LOWFS_Im_matrix_num_LO_15.fits'))
G_OBWFS_real = fits.getdata(os.path.join(savedpath, 'matrix_numerical', 'EFIELD_OBWFS_Re_matrix_num_MID_6.fits'))
G_OBWFS_imag = fits.getdata(os.path.join(savedpath, 'matrix_numerical', 'EFIELD_OBWFS_Im_matrix_num_MID_6.fits'))

In [ ]:
print(G_LO_real.shape, G_LO_imag.shape, G_MID_real.shape, G_MID_imag.shape, G_HI_real.shape, G_HI_imag.shape)
#print(G_LOWFS_real.shape, G_LOWFS_imag.shape, G_OBWFS_imag.shape, G_OBWFS_real.shape)


nth_lo  = G_LO_real.shape[0]      # value same as max_LO taken from config.ini file
nth_mid = G_MID_real.shape[0]     # 120*max_MID
nth_hi  = G_HI_real.shape[0]      # square of max_HI 


print("\n","*****************      LOW_ORDER field imaginary and real part    **************","\n")
fig = plt.figure(figsize=(20,10))
for i in range(0, nth_lo):
    plt.subplot(3,5,i+1)
    im = plt.imshow(np.log(np.abs(np.reshape(G_LO_real[i], [227,227]))), vmin=-30, vmax=-5)
    plt.title("G_LO_real_%d"%i)
    plt.colorbar(im)
plt.tight_layout()


fig = plt.figure(figsize=(20,10))
for i in range(0, nth_lo):
    plt.subplot(3,5,i+1)
    im = plt.imshow(np.log(np.abs(np.reshape(G_LO_imag[i], [227,227]))), vmin=-30, vmax=-5)
    plt.title("G_LO_imag_%d"%i)
    plt.colorbar(im)
plt.tight_layout()


# print("\n","*****************      MID_ORDER field imaginary and real part    **************","\n")
# fig = plt.figure(figsize=(40,20))
# for i in range(0, nth_mid):
#     plt.subplot(20,36,i+1)
#     im = plt.imshow(np.log(np.abs(np.reshape(G_MID_real[i], [227,227]))), vmin=-30, vmax=-5)
#     plt.title("G_MID_real_%d"%i)
#     plt.colorbar(im)
# #plt.tight_layout()


# fig = plt.figure(figsize=(40,20))
# for i in range(0, nth_mid):
#     plt.subplot(20,36,i+1)
#     im = plt.imshow(np.log(np.abs(np.reshape(G_MID_imag[i], [227,227]))), vmin=-30, vmax=-5)
#     plt.title("G_MID_imag_%d"%i)
#     plt.colorbar(im)
# plt.tight_layout()



In [ ]:
design = 'medium'

In [ ]:
overall_dir = util.create_data_path(CONFIG_PASTIS.get('local', 'local_data_path'), telescope = 'luvoir-'+design)
os.makedirs(overall_dir, exist_ok=True)
resDir = os.path.join(overall_dir, 'matrix_numerical')

# # Moving parts parameters
max_LO = CONFIG_PASTIS.getint('calibration', 'max_LO')
max_MID = CONFIG_PASTIS.getint('calibration', 'max_MID')
max_HI = CONFIG_PASTIS.getint('calibration', 'max_HI')
num_DM_act = CONFIG_PASTIS.getint('calibration', 'num_DM_act')

# # General telescope parameters
nb_seg = CONFIG_PASTIS.getint('LUVOIR', 'nb_subapertures')
wvln = CONFIG_PASTIS.getfloat('LUVOIR', 'lambda') * 1e-9  # m
diam = CONFIG_PASTIS.getfloat('LUVOIR', 'diameter')  # m
nm_aber = CONFIG_PASTIS.getfloat('calibration', 'single_aberration') * 1e-9   # m

# # Image system parameters
im_lamD = CONFIG_PASTIS.getfloat('numerical', 'im_size_lamD_hcipy')  # image size in lambda/D
sampling = CONFIG_PASTIS.getfloat('numerical', 'sampling')

In [ ]:
optics_input = CONFIG_PASTIS.get('LUVOIR', 'optics_path')
#luvoir = LuvoirAPLC(optics_input, design, sampling)

In [ ]:
luvoir = LuvoirAPLC(optics_input, design, sampling)
luvoir_a = LuvoirA_APLC(optics_input, design, sampling)

In [ ]:
#luvoir.seg_diameter_circumscribed, 
luvoir_a.segment_circumscribed_diameter

In [ ]:
luvoir_a.apod_design, luvoir_a.apod_dict, luvoir.fpm, luvoir.pupil_grid

In [ ]:
segment = hc.hexagonal_aperture(luvoir.segment_circumscribed_diameter, np.pi / 2) #function)
segment_sampled = hc.evaluate_supersampled(segment,luvoir.pupil_grid, 1) #field

plt.figure(figsize=(10,5))

plt.subplot(1,2,1)
plt.title("segment_sampled")
hc.imshow_field(segment_sampled)

plt.subplot(1,2,2)
plt.title("luvoir.pupil_grid")
plt.plot(luvoir.pupil_grid.x, luvoir.pupil_grid.y, '+')
#plt.axis('equal')
plt.xlabel('x')
plt.ylabel('y')

plt.show()

In [ ]:
aper2, segs2 = hc.make_segmented_aperture(segment,luvoir_a.seg_pos, segment_transmissions=1, return_segments=True)
luvoir_segmented_pattern = hc.evaluate_supersampled(aper2, luvoir.pupil_grid, 1)
seg_evaluated = []
for seg_tmp in segs2:
    tmp_evaluated = hc.evaluate_supersampled(seg_tmp, luvoir.pupil_grid, 1)
    seg_evaluated.append(tmp_evaluated)
    
plt.figure()
hc.imshow_field(luvoir_segmented_pattern)
plt.show()

In [ ]:
luvoir.make_LO_Modes(max_LO)

In [ ]:
global_zernike_basis = hc.mode_basis.make_zernike_basis(Nzernike_global,self.diam, self.pupil_grid,
                                                                starting_mode=1, ansi=False, radial_cutoff=True, use_cache=True)